In [6]:

import sys
import math
import time
from random import randint,choice, shuffle, random
from statistics import mode
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
# from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import *
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix 

def specificity(y_true, y_predicted):
 tp,fn,fp,tn = confusion_matrix(y_true, y_predicted).ravel()
 s=tn/(tn+fp)
 return s

my_scorer = make_scorer(specificity, greater_is_better=True)





In [10]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/M_dataset/creditcardM.csv')
data.drop('Time',axis=1,inplace=True)


In [11]:

data[(data['Class']==1)].index

Int64Index([  25,   40,   51,   58,   82,   98,  123,  150,  160,  178,
            ...
            9844, 9849, 9861, 9907, 9914, 9922, 9927, 9948, 9968, 9979],
           dtype='int64', length=492)

In [12]:
col_names=list(data.columns)
Att_names=col_names[:-1]
Class_names=data["Class"].unique()

In [13]:
n_att = len(Att_names)
n_att


 

29

In [14]:

train_data, test_data = train_test_split(data, test_size=0.3)
train_x=train_data.iloc[:,:-1].values
train_y=train_data["Class"].values
test_x=test_data.iloc[:,:-1].values
test_y=test_data["Class"].values


LogisticRegression


In [15]:

# Logestic LogisticRegression

clf=GridSearchCV(LogisticRegression(max_iter=100,n_jobs=-1,class_weight='balanced'),scoring=my_scorer,param_grid={'C':[0.1]})






In [16]:
#Constants
#pheromone concentration influence 
alpha = 0.5 
#MI influence
beta = 0.5
#pheromone evoparation
rho = 0.1
#Pheromone diposite level
q = 0.5
epsilon = 0.001

In [17]:
clf.fit(train_x,train_y)
clf.score(test_x,test_y)


0.8843537414965986

In [18]:

# number of ants and number of iterations
tset = list(range(n_att))
n_ants = 10
n_iter = 10
ants=[[] for x in range(n_ants)]
ac_ants=[0 for x in range(n_ants)]
best_tour = tset.copy()
best_ac=clf.fit(train_x[:,best_tour],train_y).score(test_x[:,best_tour],test_y)
#Intializing Pheromone
ph_trail = []
for i in range(n_att):
    p_r = [0.5 for j in range(n_att)]
    p_r[i] = 0
    ph_trail.append(p_r)
t_time_s = time.time()
t_t_time = 0
for i in range(n_iter):
    print("Iteration No :", i)
    at_time_s = time.time()
    for a in range(n_ants):
        ants[a]=[choice(best_tour)]
        while True:
            change=False
            mic = clf.fit(train_x[:,ants[a]],train_y).score(test_x[:,ants[a]],test_y)
            next_pos = {}
            max_next = []
            for at in range(n_att):
                if at not in ants[a]:
                    ph=0
                    hi = clf.fit(train_x[:,ants[a]+[at]],train_y).score(test_x[:,ants[a]+[at]],test_y)- mic
                    hi= 0 if hi<=0 else hi
                    for x_pos in ants[a]:
                        ph+=ph_trail[x_pos][at]
                    ph=ph/len(ants[a])
                    t_value = (hi ** beta) * (ph ** alpha)
                    if t_value>0:
                        max_next.append((t_value,at))
            if(len(max_next)):    
                pr=randint(0,len(max_next)-1)
                n_at = max_next[randint(0, pr)][1]
                ants[a].append(n_at)
                change=True
            if change==False:
                break
    for a in range(n_ants):
        ascore=clf.fit(train_x[:,ants[a]],train_y).score(test_x[:,ants[a]],test_y)
        ac_ants[a]=(ascore)
        bscore=best_ac
        if ascore>bscore:
            best_tour = ants[a].copy()
            best_ac=ac_ants[a]
        elif ascore==bscore:
            if len(ants[a])<len(best_tour):
                best_tour = ants[a].copy()
                best_ac=ac_ants[a]
    for a in range(n_ants):
        for k in ants[a]:
            for j in ants[a]:
                if k != j:
                    ph_trail[k][j] += ph_trail[k][j] *  ac_ants[a]/ len(ants[a])
                    if ph_trail[k][j]>1:
                        ph_trail[k][j]=1
    for k in best_tour:
        for j in best_tour:
            if k != j:
                ph_trail[k][j] += ph_trail[k][j] *  best_ac/ len(ants[a])
                if ph_trail[k][j]>1:
                    ph_trail[k][j]=1
    for j in range(n_att):
        for k in range(n_att):
            if k!=j:
                ph_trail[j][k]*=(1-rho)
    at_time_e = time.time()
    tempset=set()
    print("###########Iteration : ",i,"############")
    for a in range(n_ants):
        print("Sol of ant ",a," is :",ants[a],"     Specificity:",ac_ants[a])
    print("Sol of best ant till now is :",[Att_names[x] for x in best_tour],"     Specificity:",best_ac)
    print("Best_Sol length :  ", len(best_tour))
    print("Time Taken by iteration ", i, " is :", at_time_e - at_time_s)
t_time_e = time.time()
print("Total time by ACO (in sec): ", t_time_e - t_time_s)
print("Best_Sol  :  ", [Att_names[x] for x in best_tour])
print("Best_Sol length :  ", len(best_tour))
print("Specificity of Best_ant solution is :",best_ac)


Iteration No : 0
###########Iteration :  0 ############
Sol of ant  0  is : [14, 2, 3, 0, 22, 25]      Specificity: 0.8979591836734694
Sol of ant  1  is : [24, 1, 4, 9, 22, 11, 3, 12, 6, 13]      Specificity: 0.8979591836734694
Sol of ant  2  is : [0, 11, 5, 3, 1]      Specificity: 0.8843537414965986
Sol of ant  3  is : [25, 2, 4, 5, 9, 10, 12, 3, 7, 6]      Specificity: 0.8979591836734694
Sol of ant  4  is : [21, 0, 3, 6, 7]      Specificity: 0.8843537414965986
Sol of ant  5  is : [23, 2, 10, 3, 5, 7]      Specificity: 0.8843537414965986
Sol of ant  6  is : [2, 3, 0, 4]      Specificity: 0.891156462585034
Sol of ant  7  is : [7, 2, 3, 0, 22, 20]      Specificity: 0.9047619047619048
Sol of ant  8  is : [14, 15, 2, 3, 12, 1, 7]      Specificity: 0.8775510204081632
Sol of ant  9  is : [6, 1, 9, 13, 3, 12, 5, 11]      Specificity: 0.8979591836734694
Sol of best ant till now is : ['V8', 'V3', 'V4', 'V1', 'V23', 'V21']      Specificity: 0.9047619047619048
Best_Sol length :   6
Time Taken by